# Run SageMaker Training Job with Your Docker Image

This code shows how to run code written in notebook as SageMaker Training Job by using sagemill <br>
In this example, we use `sagemaker.estimator.Estimator` as Estimator

#### assumptions
- this notebook is from "conda_python3" in SageMaker notebook instance

In [ ]:
!conda install -y scikit-learn

## Parameters
The cell with "parameters" tag is used as hyperparameters in SageMaker Training Job. <br>
So these can be overwritten by `hyperparameters` in `sagemaker.estimator.Estimator`

#### How to add tags
1. Click 'View' tab on the notebook
2. Click 'Cell Toolbar'
3. Click 'Tags'
4. Input tag name to the cell
5. Click "Add tag"

In [ ]:
data_dir = './dataset'
model_dir = './model'
# for iris
num_class = 3
dim_data = 4

In [ ]:
from sklearn.svm import SVC
import os
import numpy as np
import pickle

x_filename = 'x.npy'
y_filename = 'y.npy'

## Download dataset & upload it to S3
The cell with "sagemaker" tag is ignored when generating python script from the notebook by `Converter.generate_pyfile`

In [ ]:
import sagemaker
from sklearn import datasets
from sklearn.model_selection import train_test_split
import shutil

iris = datasets.load_iris()

# In local, save only sample for debugging
os.makedirs(data_dir, exist_ok=True)
x_train, _, y_train, _ = train_test_split(iris.data, iris.target, train_size=100)
np.save(os.path.join(data_dir, x_filename), x_train)
np.save(os.path.join(data_dir, y_filename), y_train)

# In S3, upload all data for full training
tmp_dir = 'tmp_dataset'
os.makedirs(tmp_dir, exist_ok=True)
np.save(os.path.join(tmp_dir, x_filename), iris.data)
np.save(os.path.join(tmp_dir, y_filename), iris.target)
s3_input = sagemaker.Session().upload_data(path=tmp_dir, key_prefix='datasets/sagemill_custom')
shutil.rmtree(tmp_dir)

## Code to train your model
You can write anything here

In [ ]:
# training code
x_train = np.load(os.path.join(data_dir, x_filename))
y_train = np.load(os.path.join(data_dir, y_filename))

clf = SVC()

clf.fit(x_train, y_train)
os.makedirs(model_dir, exist_ok=True)
with open(os.path.join(model_dir, 'my_model.sav'), 'wb') as f:
    pickle.dump(clf, f)

## Prepare your docker image for training job

`builder.create_template` creates the following files. <br>
Edit the files to customize your image

**`entrypoint.py` and `requirements.txt` are generated by `builder.build_image` later**

```
docker
├── Dockerfile
└── entrypoint.sh
```

In [ ]:
from sagemill import TrainImageBuilder

builder = TrainImageBuilder(dest_dir='docker')
builder.create_template()

## Run SageMaker Training Job
### `builder.build_image` builds your docker image
- This generates `docker/entrypoint.py` from this notebook
- This generates `requirements.txt` by running `pipreqs` to the `entrypoint.py`
- If you created your own `requirements.txt`, set `skip_generate_requirements=True`
- **Save this notebook file before runnning the following cell**

### `builder.push_image` pushes your docker image to ECR
- The IAM role attached to your notebook instance needs to have permission to do
  - ECR
    - create-repository
    - get-authorization-token
  - STS
    - get-caller-identity

### `estimator.fit` run the training job
- `hyper_params` corresponds with the cell tagged "parameters" <br>
  In the training job, the parameters are overwritten to the values of `hyper_params`


In [ ]:
from sagemaker.estimator import Estimator

role = sagemaker.get_execution_role()
max_run_time = 24 * 60 * 60 * 1  # 1 day
hyper_params = {
    'model_dir': '/opt/ml/model',
    'data_dir': '/opt/ml/input/data/training',
}

image_name = builder.build_image(notebook='./train_custom.ipynb',
                                 image='sagemill',
                                 tag='0.1',
                                 skip_generate_requirements=False)
builder.push_image(image_name)

estimator = Estimator(
    image_name=image_name,
    role=role,
    train_instance_count=1,
    train_instance_type='ml.c5.xlarge',
    train_max_run=max_run_time,
    hyperparameters=hyper_params)

estimator.fit(inputs=s3_input)